In [46]:
# import library
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

%matplotlib inline 

In [47]:
# Data Loading
sales_train = pd.read_csv('data/sales_train.csv', parse_dates=[0])
#sales_train_df = pd.read_csv('data/sales_train_df.csv', parse_dates=[0])
items = pd.read_csv('data/items.csv')
shops = pd.read_csv('data/shops.csv')
item_categories = pd.read_csv('data/item_categories.csv')
#test = pd.read_csv('data/test.csv')
#sample_submission = pd.read_csv('data/sample_submission.csv')

In [48]:
item_categories["item_category_name"] = item_categories["item_category_name"].apply(lambda x: x.lower()).str.replace('[^\w\s]', '').str.replace('\d+','').str.strip()
item_categories['section'] = item_categories['item_category_name'].str.partition(' ')[0].fillna('NO_DATA')
item_categories['main_category'] = item_categories['item_category_name'].str.partition(' ')[2].fillna('NO_DATA')
item_categories['sub_category'] = item_categories['main_category'].str.partition(' ')[0]
item_categories

,item_category_name,item_category_id,section,main_category,sub_category
0,pc гарнитурынаушники,0,pc,гарнитурынаушники,
1,аксессуары ps,1,аксессуары,ps,
2,аксессуары ps,2,аксессуары,ps,
3,аксессуары ps,3,аксессуары,ps,
4,аксессуары psp,4,аксессуары,psp,
5,аксессуары psvita,5,аксессуары,psvita,
6,аксессуары xbox,6,аксессуары,xbox,
7,аксессуары xbox one,7,аксессуары,xbox one,
8,билеты цифра,8,билеты,цифра,цифра
9,доставка товара,9,доставка,товара,товара


In [49]:
shops['shop_name'] = shops['shop_name'].apply(lambda x: x.lower()).str.replace('[^\w\s]', '').str.replace('\d+','').str.strip()
shops['shop_city'] = shops['shop_name'].str.partition(' ')[0]
shops['shop_type'] = shops['shop_name'].apply(lambda x: 'мтрц' if 'мтрц' in x else 'трц' if 'трц' in x else 'трк' if 'трк' in x else 'тц' if 'тц' in x else 'тк' if 'тк' in x else 'NO_DATA')
shops.head()

,shop_name,shop_id,shop_city,shop_type
0,якутск орджоникидзе фран,0,якутск,NO_DATA
1,якутск тц центральный фран,1,якутск,тц
2,адыгея тц мега,2,адыгея,тц
3,балашиха трк октябрькиномир,3,балашиха,трк
4,волжский тц волга молл,4,волжский,тц


In [50]:
# sales_train_df = sales_train.copy()
#plt.figure(figsize=(10,4))
#plt.xlim(-100, 3000)
#sns.boxplot(x=sales_train_df['item_cnt_day'])

#plt.figure(figsize=(10,4))
#plt.xlim(sales_train_df['item_price'].min(), sales_train_df['item_price'].max())
#sns.boxplot(x=sales_train_df['item_price'])

In [75]:
sales_train_df = sales_train.copy()
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [76]:
from tqdm import tqdm
new_features = []
lookback_range = list(range(1, 33 + 1))
groups = sales_train.groupby(by = ['item_id', 'date_block_num'])
for diff in tqdm(lookback_range):
    feature_name = 'prev_item_price_' + str(diff)
    result = groups.agg({'item_price':'mean'})
    result = result.reset_index()
    result.loc[:, 'date_block_num'] += diff
    result.rename(columns={'item_price': feature_name}, inplace=True)
    sales_train_df = sales_train_df.merge(result, on = ['item_id', 'date_block_num'], how = 'inner')
    sales_train_df[feature_name] = sales_train_df[feature_name]
    new_features.append(feature_name)



  0%|          | 0/33 [00:00<?, ?it/s]

  3%|▎         | 1/33 [00:01<00:38,  1.20s/it]

  6%|▌         | 2/33 [00:02<00:36,  1.18s/it]

  9%|▉         | 3/33 [00:03<00:31,  1.05s/it]

 12%|█▏        | 4/33 [00:03<00:25,  1.14it/s]

 15%|█▌        | 5/33 [00:04<00:21,  1.32it/s]

 18%|█▊        | 6/33 [00:04<00:18,  1.49it/s]

 21%|██        | 7/33 [00:04<00:15,  1.66it/s]

 24%|██▍       | 8/33 [00:05<00:13,  1.83it/s]

 27%|██▋       | 9/33 [00:05<00:12,  1.99it/s]

 30%|███       | 10/33 [00:06<00:10,  2.12it/s]

 33%|███▎      | 11/33 [00:06<00:09,  2.31it/s]

 36%|███▋      | 12/33 [00:06<00:08,  2.41it/s]

 39%|███▉      | 13/33 [00:07<00:10,  1.96it/s]

 42%|████▏     | 14/33 [00:08<00:09,  1.96it/s]

 45%|████▌     | 15/33 [00:08<00:09,  1.87it/s]

 48%|████▊     | 16/33 [00:09<00:08,  2.09it/s]

 52%|█████▏    | 17/33 [00:09<00:07,  2.15it/s]

 55%|█████▍    | 18/33 [00:09<00:05,  2.55it/s]

 58%|█████▊    | 19/33 [00:09<00:04,  2.87it/s]

 61%|██████    | 20/33 [00:10<00:04,

In [77]:
#sales_train_df["item_cnt_day"] = sales_train_df["item_cnt_day"].clip(0, 20)
#sales_train_df["item_price"] = sales_train_df["item_price"].clip(0, 5000)
## Date Attribute

sales_train_df['item_cnt_day'] = sales_train_df['item_cnt_day'].clip(0,20)

sales_train_df["year"] = pd.to_datetime(sales_train_df["date"]).dt.year
sales_train_df["month"] = pd.to_datetime(sales_train_df["date"]).dt.month

sales_item_sum = sales_train_df.groupby(["shop_id","item_id","year","month"], as_index=False)['item_cnt_day'].sum()
sales_item_sum = sales_item_sum.rename(columns={'item_cnt_day': 'item_cnt_month'})

sales_item_sum['item_cnt_month'] = sales_item_sum['item_cnt_month'].clip(0,20)
sales_train_df =  pd.merge(sales_train_df, sales_item_sum, on=["shop_id","item_id","year","month"], how='inner')

sales_item_avg = pd.DataFrame(sales_train_df.groupby(["shop_id","item_id","year","month"], as_index=False)['item_cnt_day'].mean())
sales_item_avg = sales_item_avg.rename(columns={'item_cnt_day': 'item_cnt_month_avg'})
sales_train_df =  pd.merge(sales_train_df, sales_item_avg, on=["shop_id","item_id","year","month"], how='inner')

sales_item_var = pd.DataFrame(sales_train_df.groupby(["shop_id","item_id","year","month"], as_index=False)['item_cnt_day'].var())
sales_item_var = sales_item_var.rename(columns={'item_cnt_day': 'item_cnt_month_var'})
sales_train_df =  pd.merge(sales_train_df, sales_item_var, on=["shop_id","item_id","year","month"], how='inner')

sales_train_df =  pd.merge(sales_train_df, items, on="item_id", how='inner')
sales_train_df =  pd.merge(sales_train_df, shops, on=["shop_id"], how='inner')
sales_train_df =  pd.merge(sales_train_df, item_categories, on=["item_category_id"], how='inner')

In [78]:
item_id_te = pd.DataFrame(sales_train_df.groupby(['item_id'], as_index=False)['item_cnt_month'].mean())
item_cat_id_te = pd.DataFrame(sales_train_df.groupby(['item_category_id'], as_index=False)['item_cnt_month'].mean())
shop_id_te = pd.DataFrame(sales_train_df.groupby('shop_id', as_index=False)['item_cnt_month'].mean())
date_block_num_te = pd.DataFrame(sales_train_df.groupby(['date_block_num'], as_index=False)['item_cnt_month'].mean())
shop_city_type_te = pd.DataFrame(sales_train_df.groupby(["shop_city","shop_type"], as_index=False)['item_cnt_month'].mean())
item_cat_main_sub_te = pd.DataFrame(sales_train_df.groupby(["main_category","sub_category"], as_index=False)['item_cnt_month'].mean())
section_te = pd.DataFrame(sales_train_df.groupby(['section'], as_index=False)['item_cnt_month'].mean())
year_te = pd.DataFrame(sales_train_df.groupby(['year'], as_index=False)['item_cnt_month'].mean())
month_te = pd.DataFrame(sales_train_df.groupby(['month'], as_index=False)['item_cnt_month'].mean())

In [79]:
item_id_te = item_id_te.rename(columns={'item_cnt_month': 'item_id_te'})
item_cat_id_te = item_cat_id_te.rename(columns={'item_cnt_month': 'item_cat_id_te'})
shop_id_te = shop_id_te.rename(columns={'item_cnt_month': 'shop_id_te'})
date_block_num_te = date_block_num_te.rename(columns={'item_cnt_month': 'date_block_num_te'})
shop_city_type_te = shop_city_type_te.rename(columns={'item_cnt_month': 'shop_city_type_te'})
item_cat_main_sub_te = item_cat_main_sub_te.rename(columns={'item_cnt_month': 'item_cat_main_sub_te'})
section_te = section_te.rename(columns={'item_cnt_month': 'section_te'})
year_te = year_te.rename(columns={'item_cnt_month': 'year_te'})
month_te = month_te.rename(columns={'item_cnt_month': 'month_te'})

In [80]:
sales_train_df =  pd.merge(sales_train_df, item_id_te, on=['item_id'], how='inner')
sales_train_df =  pd.merge(sales_train_df, item_cat_id_te, on=["item_category_id"], how='inner')
sales_train_df =  pd.merge(sales_train_df, shop_id_te, on=["shop_id"], how='inner')
sales_train_df =  pd.merge(sales_train_df, date_block_num_te, on=["date_block_num"], how='inner')
sales_train_df =  pd.merge(sales_train_df, shop_city_type_te, on=["shop_city","shop_type"], how='inner')
sales_train_df =  pd.merge(sales_train_df, item_cat_main_sub_te, on=["main_category","sub_category"], how='inner')
sales_train_df =  pd.merge(sales_train_df, section_te, on=["section"], how='inner')
sales_train_df =  pd.merge(sales_train_df, year_te, on=["year"], how='inner')
sales_train_df =  pd.merge(sales_train_df, month_te, on=["month"], how='inner')


In [81]:
sales_train_df.to_csv('data/sales_train_df_last.csv', index=None)

In [82]:
feature_col = [
    "item_id_te",
    "shop_id_te",
    "date_block_num_te",
    "item_cat_id_te",
    "item_price",
    "shop_city_type_te",
    "item_cat_main_sub_te",
    "section_te",
    "year_te",
    "month_te",
    "item_cnt_month_var",
    "item_cnt_month"] + new_features

supurvised_col = ["item_cnt_month_avg"]

# split data for crossvalidation
X_train, X_test, y_train, y_test = train_test_split(sales_train_df[feature_col], sales_train_df[supurvised_col], test_size=0.2, random_state=1)

In [83]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
import xgboost as xgb

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(sales_train_df[feature_col])
X_new_columns = X_train.columns.values
X_train = pd.DataFrame(imp.transform(sales_train_df[feature_col]), columns=X_new_columns)
X_test = pd.DataFrame(imp.transform(X_test), columns=X_new_columns)

# パイプラインのセット
pipe_xgb = Pipeline([('scl',StandardScaler()),('est',xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 16))])

# 学習
pipe_xgb.fit(X_train, y_train.as_matrix().ravel())
#pipe_randomFore.fit(X_train_all, sales_train_df[supurvised_col])

/Users/ootakeitasuku/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


XGBoostError: b'[00:34:10] src/objective/regression_obj.cc:44: Check failed: preds->Size() == info.labels_.size() (5989 vs. 4791) labels are not correctly providedpreds.size=5989, label.size=4791\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000001ada0ff806 dmlc::StackTrace() + 246\n[bt] (1) 1   libxgboost.dylib                    0x0000001ada0ff5bf dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x0000001ada17c891 xgboost::obj::RegLossObj<xgboost::obj::LinearSquareLoss>::GetGradient(xgboost::HostDeviceVector<float>*, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 593\n[bt] (3) 3   libxgboost.dylib                    0x0000001ada0fbd54 xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 996\n[bt] (4) 4   libxgboost.dylib                    0x0000001ada1180cc XGBoosterUpdateOneIter + 172\n[bt] (5) 5   libffi.6.dylib            

In [ ]:
from sklearn.metrics import mean_squared_error

#評価
y_train_pred = pipe_xgb.predict(X_train)
y_test_pred = pipe_xgb.predict(X_test)
print('MSE train data: ', mean_squared_error(y_train, y_train_pred)) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data: ', mean_squared_error(y_test, y_test_pred))         # 検証データを用いたときの平均二乗誤差を出力

In [ ]:
from sklearn.metrics import r2_score

# 評価
print('r^2 train data: ', r2_score(y_train, y_train_pred))
print('r^2 test data: ', r2_score(y_test, y_test_pred))

In [ ]:
sales_train_df.shape

In [ ]:
item_price = sales_train_df[["item_id","item_price","item_cnt_month_var","item_cnt_month_avg","item_cnt_month"]]
item_price = item_price.drop_duplicates(subset='item_id', keep='last')

In [ ]:
items = pd.read_csv('data/items.csv')
sales_test_df = pd.read_csv('data/test.csv')
sales_test_df =  pd.merge(sales_test_df, items, on=["item_id"], how='left')
sales_test_df =  pd.merge(sales_test_df, item_price, on="item_id", how='left')
sales_test_df =  pd.merge(sales_test_df, shops, on=["shop_id"], how='left')
sales_test_df =  pd.merge(sales_test_df, item_categories, on=["item_category_id"], how='left')
sales_test_df["date_block_num"] = 34
sales_test_df["year"] = 2015
sales_test_df["month"] = 11
sales_test_df =  pd.merge(sales_test_df, item_id_te, on=["item_id"], how='left')
sales_test_df =  pd.merge(sales_test_df, shop_id_te, on=["shop_id"], how='left')
sales_test_df =  pd.merge(sales_test_df, item_cat_id_te, on=["item_category_id"], how='left')
sales_test_df =  pd.merge(sales_test_df, date_block_num_te, on=["date_block_num"], how='left')
sales_test_df =  pd.merge(sales_test_df, shop_city_type_te, on=["shop_city","shop_type"], how='left')
sales_test_df =  pd.merge(sales_test_df, item_cat_main_sub_te, on=["main_category","sub_category"], how='left')
sales_test_df =  pd.merge(sales_test_df, section_te, on=["section"], how='left')
sales_test_df =  pd.merge(sales_test_df, year_te, on=["year"], how='left')
sales_test_df =  pd.merge(sales_test_df, month_te, on=["month"], how='left')

In [ ]:
from tqdm import tqdm
new_features = []
lookback_range = list(range(1, 33 + 1))
groups = sales_train.groupby(by = ['item_id', 'date_block_num'])
for diff in tqdm(lookback_range):
    feature_name = 'prev_item_price_' + str(diff)
    result = groups.agg({'item_price':'mean'})
    result = result.reset_index()
    result.loc[:, 'date_block_num'] += diff
    result.rename(columns={'item_price': feature_name}, inplace=True)
    sales_test_df = sales_test_df.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
    sales_test_df[feature_name] = sales_test_df[feature_name]
    new_features.append(feature_name)


In [ ]:
sales_test_df.T

In [ ]:
 sales_test_df.to_csv('data/sales_test_df2.csv')

In [ ]:
sales_test_df = pd.DataFrame(imp.transform(sales_test_df[X_new_columns]), columns=X_new_columns)

In [ ]:
sales_test_df.shape

In [ ]:
y_test_pred = pipe_xgb.predict(sales_test_df[feature_col])
y_test_pred = pd.DataFrame(y_test_pred)

In [ ]:
y_test_pred = y_test_pred.reset_index()
ID = test["ID"].reset_index()

In [ ]:
y_test_pred =  pd.merge(ID, y_test_pred, on=["index"], how='inner')

In [ ]:
y_test_pred = y_test_pred.drop(columns="index", axis=0)

In [ ]:
y_test_pred = y_test_pred.rename(columns={0: 'item_cnt_month'})

In [ ]:
y_test_pred.to_csv('data/submit_v5.csv', index=None)

In [ ]:
y_test_pred